# Example Notebook

### Using the 01_generate_single_synth_parameter_data.yaml experiment

This notebook is meant to explain how the objects in this class work, and are configurable in a notebook setting. 

Notebooks are a replacement for the `Experiment` class, as we will be handling our experiments in the notebook setting rather than using a .py file

First, let's import all of the stuff we need

In [10]:
# Python Lib Packages
import os
from pathlib import Path
import sys

# Pypi imported Modules
import hydra
import matplotlib.pyplot as plt
import numpy as np
from omegaconf import DictConfig, OmegaConf
import pandas as pd
import torch
import torch.nn as nn

# Putting the dMC module in the Python Path
current_dir = Path.cwd()
dmc_dev_path = current_dir.parents[0]
sys.path.append(str(dmc_dev_path))

# Synthetic Parameter distributions and MLP Networks
from dMC.nn.power_distribution import Power
from dMC.nn.single_parameters import SingleParameters
from dMC.nn.inverse_linear import InverseLinear
from dMC.nn.parameter_list import ParameterList
from dMC.nn.mlp import MLP
from dMC.nn import Initialization

# Physics model
from dMC.physics.explicit_mc import ExplicitMC

# Experiment
from dMC.experiments.generate_synthetic import GenerateSynthetic
from dMC.experiments.writer import Writer

# Utils functions
from dMC.__main__ import _set_defaults

# Required to generate data
from dMC.data.datasets.nhd_srb import NHDSRB
from dMC.data.observations.usgs import USGS
from dMC.data.dates import Dates
from dMC.data.normalize.min_max import MinMax
from dMC.data import DataLoader

# For evaluation
from dMC.experiments.metrics import Metrics

## Setting up the Config

Let's import the config files from our `dMC` directory:

**Note** this uses the config settings from `../dMC/conf/gloal_settings.yaml`

In [11]:
config_path = "../dMC/conf/"
with hydra.initialize(config_path=config_path, version_base="1.3"):
    cfg = hydra.compose(config_name="global_settings.yaml", return_hydra_config=True)

cfg = _set_defaults(cfg)
cfg

{'hydra': {'run': {'dir': 'outputs/${now:%Y-%m-%d}/${now:%H-%M-%S}'}, 'sweep': {'dir': 'multirun/${now:%Y-%m-%d}/${now:%H-%M-%S}', 'subdir': '${hydra.job.num}'}, 'launcher': {'_target_': 'hydra._internal.core_plugins.basic_launcher.BasicLauncher'}, 'sweeper': {'_target_': 'hydra._internal.core_plugins.basic_sweeper.BasicSweeper', 'max_batch_size': None, 'params': None}, 'help': {'app_name': '${hydra.job.name}', 'header': '${hydra.help.app_name} is powered by Hydra.\n', 'footer': 'Powered by Hydra (https://hydra.cc)\nUse --hydra-help to view Hydra specific help\n', 'template': '${hydra.help.header}\n== Configuration groups ==\nCompose your configuration from those groups (group=option)\n\n$APP_CONFIG_GROUPS\n\n== Config ==\nOverride anything in the config (foo.bar=value)\n\n$CONFIG\n\n${hydra.help.footer}\n'}, 'hydra_help': {'template': "Hydra (${hydra.runtime.version})\nSee https://hydra.cc for more info.\n\n== Flags ==\n$FLAGS_HELP\n\n== Configuration groups ==\nCompose your configura

# Building Objects

Below we'll do the "behind the scenes" work of building our Dataloader, Model, and Experiment so that we can just use those objects here

## Dataloader:

In [12]:
cfg_data = cfg.config.data

dates = Dates(cfg_data)  # Dates Object
normalize = MinMax(cfg_data)  # Normalization Object
data = NHDSRB(cfg_data, dates=dates, normalize=normalize)  # Dataset Object
obs = USGS(cfg_data, dates, normalize)  # Observations Object

# Getting the data
hydrofabric = data.get_data()
observations = obs.get_data().transpose(0, 1)

dataloader =  DataLoader(data, obs)(cfg_data)
dataloader

## Model:

In [13]:
cfg_model = cfg.config.model

neural_network = SingleParameters(cfg=cfg_model).to(cfg_model.device)
# neural_network = Power(cfg=cfg_model).to(cfg_model.device)
# neural_network = MLP(cfg=cfg_model).to(cfg_model.device)
# ... etc (see imports for all options)

physics_model = ExplicitMC(cfg=cfg_model, neural_network=neural_network)
physics_model

ExplicitMC(
  (neural_network): SingleParameters()
)

In [14]:
physics_model.neural_network.n

Parameter containing:
tensor(0.0300, requires_grad=True)

## Experiment:

In [15]:
cfg_experiment = cfg.config.experiment
writer = Writer(cfg_experiment)
experiment = GenerateSynthetic(cfg=cfg_experiment, writer=writer)
experiment

# Running the experiment

Similar to the dependency injection framework in the code, you can run the experiment like below

In [16]:
experiment.run(dataloader, physics_model)

Epoch 0: Explicit Muskingum Cunge Routing:   0%|          | 0/1343 [00:00<?, ?it/s]

In [19]:
experiment.save_path

PosixPath('/data/tkb5476/projects/dMC-Juniata-hydroDL2/runs/01_synthetic_data')

In [20]:
# Our synthetic discharge outputs. The Rows represent time, the Cols are the edge associated with the discharge
df = pd.read_csv(experiment.save_path / "01_generate_single_synth_parameter_data.csv")
df.head()

Unnamed: 0      1053      1280      2662      2689      2799      4780  \
0           0  0.072551  0.303997  0.035397  0.017339  0.273754  0.097784   
1           1  0.345283  1.010940  0.064402  0.048944  0.513326  0.258317   
2           2  0.755540  1.308555  0.099312  0.127346  0.782226  0.438819   
3           3  1.101563  1.415994  0.155389  0.183362  1.098740  0.659021   
4           4  1.485405  1.499759  0.235011  0.221742  1.618123  0.993112   

       4801      4809  
0  0.170211  0.272679  
1  0.222125  0.407868  
2  0.284994  0.669957  
3  0.400763  0.938000  
4  0.632635  1.268810

# What now?

Feel free to check out the other experiments. All of the objects that they use are included in their `service_locator` config entry

### 01: Single Parameter Experiments
To run these, you should use the following configs:
- `01_generate_single_synth_parameter_data.yaml`
- `01_train_against_single_synthetic.yaml`

### 02: Synthetic Parameter Distribution Recovery

There are many synthetic parameter experiments. Run the following configs to recreate them

#### Synthetic Constants
- `02_generate_mlp_param_list.yaml`
- `02_train_mlp_param_list.yaml`

#### Synthetic Power Law A
- `02_generate_mlp_power_a.yaml`
- `02_train_mlp_power_a.yaml`

#### Synthetic Power Law B
- `02_train_mlp_power_b.yaml`
- `02_generate_mlp_power_b.yaml`

### 03: Train against USGS data:
You can run the following cfgs to train models against USGS data
- `03_train_usgs_period_1a.yaml`
- `03_train_usgs_period_1b.yaml`
- `03_train_usgs_period_2a.yaml`
- `03_train_usgs_period_2b.yaml`
- `03_train_usgs_period_3a.yaml`
- `03_train_usgs_period_3b.yaml`
- `03_train_usgs_period_4a.yaml`
- `03_train_usgs_period_4b.yaml`